In [9]:
import vtk
import networkx as nx

import os 

import ast
import numpy as np

#from writeNodesEdges import writeObjects
from config.writeMotifs import writeNetworksMotifs

In [2]:
# ---------------------------SETUP AND COLLECTION OF QUAKES------------------------------------#
# Which region do you want to analyze ?
region = input('Input region : vrancea / romania / california / italy / japan : ')

# The side of the cubes that you split the region in 
side = int(input('Input side of the cube split in km 5 / 10 / 20 - recommended = 5 : '))

# Select desired magnitude threshold
mag = int(input('Select desired magnitude threshold - recommended = 4 : '))

G = nx.read_gexf(f'./results/{region}/networks/view/network{region}_{side}km_{mag}mag.gexf')


motif = input('Input motif: Triangles (for areas) / Tetrahedrons (for volumes) : ')

In [3]:
# Use readlines() to open the 
fileMotif = open(f'./results/{region}/networks/motifs/motifs{motif}_network{region}_{side}km_{mag}mag.txt', 'r')
linesMotif = fileMotif.readlines()

In [4]:
# Properly evaluate the Lines to get the Lists
motifNodes=[]
for item in linesMotif:
    motifNodes.append(ast.literal_eval(item))

In [5]:
motifNodes

[['39', '3', '13'],
 ['76', '13', '51'],
 ['77', '54', '13'],
 ['49', '26', '25'],
 ['86', '40', '43'],
 ['98', '42', '52'],
 ['38', '3', '23'],
 ['14', '43', '1'],
 ['14', '13', '1'],
 ['105', '43', '1'],
 ['105', '13', '1'],
 ['92', '3', '54'],
 ['10', '32', '52'],
 ['64', '13', '51'],
 ['27', '58', '13'],
 ['79', '80', '13'],
 ['79', '53', '13'],
 ['78', '54', '13'],
 ['78', '53', '13'],
 ['78', '53', '54'],
 ['69', '68', '19'],
 ['41', '40', '32'],
 ['41', '42', '32'],
 ['96', '34', '52'],
 ['96', '47', '52'],
 ['96', '47', '55'],
 ['63', '40', '43'],
 ['101', '52', '25'],
 ['101', '54', '25'],
 ['101', '54', '52'],
 ['71', '22', '13'],
 ['71', '32', '22'],
 ['88', '42', '32'],
 ['66', '53', '54'],
 ['83', '54', '25'],
 ['82', '20', '13'],
 ['16', '32', '43'],
 ['85', '22', '51'],
 ['85', '22', '52'],
 ['85', '40', '51'],
 ['85', '40', '52'],
 ['47', '46', '25'],
 ['47', '52', '25'],
 ['47', '55', '46'],
 ['90', '52', '25'],
 ['93', '52', '1'],
 ['93', '58', '1'],
 ['93', '58', '52

In [6]:
if motif == 'Triangles':    
    # Graph containing triangles only
    H = nx.Graph()
    for item in motifNodes:
        H.add_edge(int(item[0]),int(item[1]))
        H.add_edge(int(item[1]),int(item[2]))
        H.add_edge(int(item[0]),int(item[2]))

else:
    # Graph containing squares only
    H = nx.Graph()
    for item in motifNodes:
        H.add_edge(int(item[0]),int(item[1]))
        H.add_edge(int(item[1]),int(item[2]))
        H.add_edge(int(item[2]),int(item[3]))
        H.add_edge(int(item[0]),int(item[3]))

    
# Set the triangle attribute = 0 to each edge
nx.set_edge_attributes(G, 0, name=motif)

In [7]:
G = nx.convert_node_labels_to_integers(G,first_label=1)

In [8]:
G.degree()

DegreeView({1: 23, 2: 2, 3: 14, 4: 2, 5: 10, 6: 2, 7: 9, 8: 4, 9: 6, 10: 4, 11: 6, 12: 2, 13: 38, 14: 4, 15: 2, 16: 6, 17: 2, 18: 16, 19: 11, 20: 9, 21: 2, 22: 15, 23: 10, 24: 5, 25: 22, 26: 8, 27: 4, 28: 2, 29: 2, 30: 2, 31: 9, 32: 23, 33: 8, 34: 11, 35: 4, 36: 13, 37: 2, 38: 3, 39: 2, 40: 16, 41: 4, 42: 8, 43: 17, 44: 2, 45: 4, 46: 9, 47: 8, 48: 2, 49: 2, 50: 2, 51: 12, 52: 30, 53: 14, 54: 20, 55: 7, 56: 2, 57: 8, 58: 9, 59: 4, 60: 2, 61: 2, 62: 6, 63: 4, 64: 4, 65: 2, 66: 5, 67: 3, 68: 4, 69: 4, 70: 2, 71: 5, 72: 2, 73: 2, 74: 2, 75: 2, 76: 2, 77: 2, 78: 4, 79: 4, 80: 2, 81: 2, 82: 6, 83: 6, 84: 2, 85: 6, 86: 2, 87: 4, 88: 5, 89: 6, 90: 6, 91: 2, 92: 4, 93: 7, 94: 2, 95: 1, 96: 4, 97: 2, 98: 2, 99: 2, 100: 2, 101: 5, 102: 2, 103: 2, 104: 2, 105: 4, 106: 4, 107: 2, 108: 2, 109: 2, 110: 2, 111: 4, 112: 2, 113: 2})

In [9]:
# Get attributes that go into VTK function

# X Y Z coords
lat =[]
long = []
depth =[]
for n in G.nodes():
    lat.append(int(G.nodes[n]['quake_xLatitude']))
    long.append(int(G.nodes[n]['quake_yLongitude']))
    depth.append(int(G.nodes[n]['quake_zDepth']))

minLat = min(lat)
maxLat = max(lat)
minLong = min(long)
maxLong = max(long)
minDepth = min(depth)
maxDepth = max(depth)
maxDimension = max(maxLat,maxLong,maxDepth)


coords=[]
for n in G.nodes():
    coords.append([ np.float32(round((int(G.nodes[n]['quake_xLatitude']) - minLat)*(maxLat/maxDimension)/(maxLat-minLat),3)),
                    np.float32(round((int(G.nodes[n]['quake_yLongitude']) - minLong)*(maxLong/maxDimension)/(maxLong-minLong),3)),
                    np.float32(round((int(G.nodes[n]['quake_zDepth']) - minDepth)*(maxDepth/maxDimension)/(maxDepth-minDepth),3))])
  
# Degree of nodes edges    
degree = [d for n, d in G.degree()]

# Weight of edges
weights = []
for (i,j) in G.edges():
    weights.append(G[i][j]['weight'])
    
# Triangle quality of edges
motifs = []
for (i,j) in G.edges():
    motifs.append(G[i][j][motif])

In [10]:
for line in coords:
    for coord in line:
        if coord < 0 or coord > 1:
            print(True)


In [11]:
motifNodes

[['39', '3', '13'],
 ['76', '13', '51'],
 ['77', '54', '13'],
 ['49', '26', '25'],
 ['86', '40', '43'],
 ['98', '42', '52'],
 ['38', '3', '23'],
 ['14', '43', '1'],
 ['14', '13', '1'],
 ['105', '43', '1'],
 ['105', '13', '1'],
 ['92', '3', '54'],
 ['10', '32', '52'],
 ['64', '13', '51'],
 ['27', '58', '13'],
 ['79', '80', '13'],
 ['79', '53', '13'],
 ['78', '54', '13'],
 ['78', '53', '13'],
 ['78', '53', '54'],
 ['69', '68', '19'],
 ['41', '40', '32'],
 ['41', '42', '32'],
 ['96', '34', '52'],
 ['96', '47', '52'],
 ['96', '47', '55'],
 ['63', '40', '43'],
 ['101', '52', '25'],
 ['101', '54', '25'],
 ['101', '54', '52'],
 ['71', '22', '13'],
 ['71', '32', '22'],
 ['88', '42', '32'],
 ['66', '53', '54'],
 ['83', '54', '25'],
 ['82', '20', '13'],
 ['16', '32', '43'],
 ['85', '22', '51'],
 ['85', '22', '52'],
 ['85', '40', '51'],
 ['85', '40', '52'],
 ['47', '46', '25'],
 ['47', '52', '25'],
 ['47', '55', '46'],
 ['90', '52', '25'],
 ['93', '52', '1'],
 ['93', '58', '1'],
 ['93', '58', '52

In [12]:

if not os.path.exists(f'./results/{region}/networksview'):
    os.makedirs(f'./results/{region}/networksview')
os.chdir(f'./results/{region}/networksview')


In [13]:

# Write the VTK file that goes in Paraview
# The network
writeNetworksMotifs(nodeCoords=coords,
                motifCoords=motifNodes,
                edges=G.edges(),
                scalar=degree, name='degree',
                scalar2=weights, name2='weight',
                escalar2=motifs, ename2=motif,
#               nodeLabel=nodeLabel,
                fileout=f'network{region}_{side}km_{mag}mag_{motif}')

In [22]:
nodeCoords=coords

In [23]:
# INITIALIZE THE NODES 
points = vtk.vtkPoints()
for node in nodeCoords:
    points.InsertNextPoint(node)

In [17]:
edges = G.edges()

In [18]:
lines = vtk.vtkCellArray()
lines.Allocate(len(edges))
for edge in edges:
    line = vtk.vtkLine()
    line.GetPointIds().SetId(0,edge[0])  
    line.GetPointIds().SetId(1,edge[1])  # line from point edge[0] to point edge[1]
    lines.InsertNextCell(line)

In [24]:
points

<vtkmodules.vtkCommonCore.vtkPoints(0x37523b0) at 0x7fa6260642e0>

In [25]:
fileout=f'network{region}_{side}km_{mag}mag_{motif}'

In [26]:
polydata = vtk.vtkPolyData()
polydata.SetPoints(points)
polydata.SetLines(lines)

writer = vtk.vtkXMLPolyDataWriter()
writer.SetFileName(fileout+'.vtp')
writer.SetInputData(polydata)
writer.Write()
    

1

In [19]:
import matplotlib.pyplot as plt
import os
import networkx as nx


from config.sqlcollect import getTable
from config.seismicZones import query
from config.network import graphCreation
from config.cubes import makeCubes

In [20]:
# ---------------------------SETUP AND COLLECTION OF QUAKES------------------------------------#
# Which region do you want to analyze ?
region = input('Input region : vrancea / romania / california / italy / japan : ')

# The side of the cubes that you split the region in 
side = int(input('Input side of the cube split in km 5 / 10 / 20 - recommended = 5 : '))

# Select desired magnitude threshold
mag = int(input('Select desired magnitude threshold - recommended = 4 : '))



#motif = input('Input motif: Triangles (for areas) / Tetrahedrons (for volumes) : ')

In [15]:
# The query for the SQL collection
sql_query= query(region)


# Magnitude window for the query that collects the database through mySQL
sql_query+=f" AND magnitude>={mag}"

# Collect the earthquakes
quakes = getTable(sql_query)

# Make the Cubes
quakes = makeCubes(quakes,region,side)

# Make the network
G = graphCreation(quakes)



/media/gabipana/hdd1/Work/PROJECTS/seismicNetworks/venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [21]:
quakes

,datetime,latitude,longitude,depth,magnitude,xLatitude,yLongitude,zDepth,cubeIndex,cubeLatitude,cubeLongitude,cubeDepth
0,1976-09-07 17:38:08.200,45.6200,26.5000,155.3,3.6,9,8,21,9821,45.5929,26.4947,152.5
1,1976-10-01 17:50:43.200,45.6800,26.4900,146.0,6.0,11,8,20,11820,45.6830,26.4947,147.5
2,1977-03-04 19:21:54.100,45.7700,26.7600,94.0,7.4,13,12,9,13129,45.7731,26.7478,92.5
3,1977-03-04 21:21:01.100,45.2200,26.6500,141.0,3.0,1,10,19,11019,45.2325,26.6213,142.5
4,1977-03-05 02:35:22.000,45.6300,26.1900,121.3,3.0,10,3,15,10315,45.6379,26.1782,122.5
...,...,...,...,...,...,...,...,...,...,...,...,...
3911,2022-03-23 21:12:45.990,45.5765,26.4791,140.0,3.4,8,8,18,8818,45.5478,26.4947,137.5
3912,2022-03-25 16:19:27.610,45.6522,26.6300,133.1,3.2,10,10,17,101017,45.6379,26.6213,132.5
3913,2022-03-30 06:48:34.830,45.5656,26.5115,130.9,3.0,8,8,17,8817,45.5478,26.4947,132.5
3914,2022-03-30 19:19:42.570,45.5642,26.4690,113.1,3.1,8,7,13,8713,45.5478,26.4314,112.5


In [16]:
nx.info(G)

/tmp/ipykernel_133693/1064119803.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(G)


'Graph with 983 nodes and 3751 edges'

In [17]:
# X Y Z coords
lat =[]
long = []
depth =[]
for n in G.nodes():
    lat.append(int(G.nodes[n]['quake_xLatitude']))
    long.append(int(G.nodes[n]['quake_yLongitude']))
    depth.append(int(G.nodes[n]['quake_zDepth']))

minLat = min(lat)
maxLat = max(lat)
minLong = min(long)
maxLong = max(long)
minDepth = min(depth)
maxDepth = max(depth)
maxDimension = max(maxLat,maxLong,maxDepth)


coords=[]
for n in G.nodes():
    coords.append([ np.float32(round((int(G.nodes[n]['quake_xLatitude']) - minLat)*(maxLat/maxDimension)/(maxLat-minLat),3)),
                    np.float32(round((int(G.nodes[n]['quake_yLongitude']) - minLong)*(maxLong/maxDimension)/(maxLong-minLong),3)),
                    np.float32(round((int(G.nodes[n]['quake_zDepth']) - minDepth)*(maxDepth/maxDimension)/(maxDepth-minDepth),3))])
  

In [18]:
# Write the VTK file that goes in Paraview
# The network
writeNetworksMotifs(nodeCoords=coords,
#                motifCoords=motifNodes,
                edges=G.edges(),
 #               scalar=degree, name='degree',
#                scalar2=weights, name2='weight',
##                escalar2=motifs, ename2=motif,
#               nodeLabel=nodeLabel,
                fileout=f'network{region}_{side}km_{mag}mag')